<a href="https://colab.research.google.com/github/willygru/chat-with-your-code-repo/blob/master/03_Module_01_Chat_with_Your_Code_LlamaIndex_and_Activeloop_Deep_Lake_for_GitHub_Repositories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index==0.9.14.post3 deeplake==3.8.8 openai==1.3.8 cohere==4.37


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.5/943.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from google.colab import userdata


In [ ]:
#import os

#os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'
#os.environ['ACTIVELOOP_TOKEN'] = '<YOUR_ACTIVELOOP_KEY>'

In [24]:
# let's setup the keys

!pip install -q python-dotenv

from dotenv import load_dotenv

load_dotenv(userdata.get('env_file_path'))

True

In [4]:
!pip install -q llama_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 11.9 MB/s eta 0:00:00


In [5]:
import os
import textwrap
from dotenv import load_dotenv
from llama_index import download_loader
from llama_hub.github_repo import GithubRepositoryReader, GithubClient
from llama_index import VectorStoreIndex
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
import re

/usr/local/lib/python3.10/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [15]:
# Fetch and set API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
active_loop_token = os.getenv("ACTIVELOOP_TOKEN")

#Set DeepLake dataset path
#dataset_path = "YOUR_DATASET_PATH"

In [ ]:
#!printenv
#!echo $HOME

In [7]:
def parse_github_url(url):
    pattern = r"https://github\.com/([^/]+)/([^/]+)"
    match = re.match(pattern, url)
    return match.groups() if match else (None, None)

def validate_owner_repo(owner, repo):
    return bool(owner) and bool(repo)

def initialize_github_client():
    github_token = os.getenv("GITHUB_TOKEN")
    return GithubClient(github_token)


In [8]:
# Check for OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise EnvironmentError("OpenAI API key not found in environment variables")

# Check for GitHub Token
github_token = os.getenv("GITHUB_TOKEN")
if not github_token:
    raise EnvironmentError("GitHub token not found in environment variables")

# Check for Activeloop Token
active_loop_token = os.getenv("ACTIVELOOP_TOKEN")
if not active_loop_token:
    raise EnvironmentError("Activeloop token not found in environment variables")

In [9]:
github_client = initialize_github_client()
download_loader("GithubRepositoryReader")

github_url = input("Please enter the GitHub repository URL: ")
owner, repo = parse_github_url(github_url)
print(owner)
print(repo)

Please enter the GitHub repository URL: https://github.com/willygru/nashville-housing
willygru
nashville-housing


In [12]:
while True:
    owner, repo = parse_github_url(github_url)
    if validate_owner_repo(owner, repo):
        loader = GithubRepositoryReader(
            github_client,
            owner=owner,
            repo=repo,
            filter_file_extensions=(
                [".sql", ".py", ".js", ".ts", ".md"],
                #[".ipynb", ".py", ".js", ".ts", ".md"],
                #[".py", ".js", ".ts", ".md"],
                GithubRepositoryReader.FilterType.INCLUDE,
            ),
            verbose=False,
            concurrent_requests=5,
        )
        print(f"Loading {repo} repository by {owner}")
        docs = loader.load_data(branch="main")
        print("Documents uploaded:")
        for doc in docs:
            print(doc.metadata)
        break  # Exit the loop once the valid URL is processed
    else:
        print("Invalid GitHub URL. Please try again.")
        github_url = input("Please enter the GitHub repository URL: ")

Loading nashville-housing repository by willygru
Documents uploaded:
{'file_path': 'README.md', 'file_name': 'README.md', 'url': 'https://github.com/willygru/nashville-housing/blob/main/README.md'}
{'file_path': 'code/SQLQueryNashvilleHousing.sql', 'file_name': 'SQLQueryNashvilleHousing.sql', 'url': 'https://github.com/willygru/nashville-housing/blob/main/code/SQLQueryNashvilleHousing.sql'}


In [29]:
print("Uploading to vector store...")

    # ====== Create vector store and upload data ======

vector_store = DeepLakeVectorStore(
    #dataset_path=userdata.get('dataset_path'),
    dataset_path='hub://your_org/repository_db',
    overwrite=True,
    runtime={"tensor_db": True},
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
query_engine = index.as_query_engine()

Uploading to vector store...
Your Deep Lake dataset has been successfully created!


Uploading data to deeplake dataset.


100%|██████████| 2/2 [00:02<00:00,  1.01s/it]
/

Dataset(path='hub://your_org/repository_db', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   


In [26]:
# Include a simple question to test.
intro_question = "What is the repository about?"
print(f"Test question: {intro_question}")
print("=" * 50)
answer = query_engine.query(intro_question)

print(f"Answer: {textwrap.fill(str(answer), 100)} \n")

Test question: What is the repository about?
Answer: The repository is about cleaning Nashville housing data. 



In [14]:
while True:
    user_question = input("Please enter your question (or type 'exit' to quit): ")
    if user_question.lower() == "exit":
        print("Exiting, thanks for chatting!")
        break

    print(f"Your question: {user_question}")
    print("=" * 50)

    answer = query_engine.query(user_question)
    print(f"Answer: {textwrap.fill(str(answer), 100)} \n")

Please enter your question (or type 'exit' to quit): What is the repository name?
Your question: What is the repository name?
Answer: The repository name is "nashville-housing". 

Please enter your question (or type 'exit' to quit): What columns were droped?
Your question: What columns were droped?
Answer: The columns that were dropped in the SQL query are OwnerAddress, TaxDistrict, PropertyAddress, and
SaleDate. 

Please enter your question (or type 'exit' to quit): Was any column updated?
Your question: Was any column updated?
Answer: Yes, columns were updated in the SQL query. 

Please enter your question (or type 'exit' to quit): Which ones?
Your question: Which ones?
Answer: The query is asking for clarification on which specific actions or steps are being referred to in
the given context. 

Please enter your question (or type 'exit' to quit): Which columns were updated?
Your question: Which columns were updated?
Answer: The columns that were updated in the SQL query are: - SaleDa

# Python main function

In [ ]:
def parse_github_url(url):
    pattern = r"https://github\.com/([^/]+)/([^/]+)"
    match = re.match(pattern, url)
    return match.groups() if match else (None, None)

def validate_owner_repo(owner, repo):
    return bool(owner) and bool(repo)

def initialize_github_client():
    github_token = os.getenv("GITHUB_TOKEN")
    return GithubClient(github_token)

def main():
    import os
    import textwrap
    from dotenv import load_dotenv
    from llama_index import download_loader
    from llama_hub.github_repo import GithubRepositoryReader, GithubClient
    from llama_index import VectorStoreIndex
    from llama_index.vector_stores import DeepLakeVectorStore
    from llama_index.storage.storage_context import StorageContext
    import re

    # Load environment variables
    load_dotenv()

    # Fetch and set API keys
    openai_api_key = os.getenv("OPENAI_API_KEY")
    active_loop_token = os.getenv("ACTIVELOOP_TOKEN")
    #dataset_path = os.getenv("DATASET_PATH")

    # Check for OpenAI API key
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise EnvironmentError("OpenAI API key not found in environment variables")

    # Check for GitHub Token
    github_token = os.getenv("GITHUB_TOKEN")
    if not github_token:
        raise EnvironmentError("GitHub token not found in environment variables")

    # Check for Activeloop Token
    active_loop_token = os.getenv("ACTIVELOOP_TOKEN")
    if not active_loop_token:
        raise EnvironmentError("Activeloop token not found in environment variables")

    github_client = initialize_github_client()
    download_loader("GithubRepositoryReader")

    github_url = input("Please enter the GitHub repository URL: ")
    owner, repo = parse_github_url(github_url)

    while True:
        owner, repo = parse_github_url(github_url)
        if validate_owner_repo(owner, repo):
            loader = GithubRepositoryReader(
                github_client,
                owner=owner,
                repo=repo,
                filter_file_extensions=(
                    #[".py", ".js", ".ts", ".md"],
                    [".sql", ".py", ".js", ".ts", ".md"],
                    #[".ipynb", ".py", ".js", ".ts", ".md"],
                    GithubRepositoryReader.FilterType.INCLUDE,
                ),
                verbose=False,
                concurrent_requests=5,
            )
            print(f"Loading {repo} repository by {owner}")
            docs = loader.load_data(branch="main")
            print("Documents uploaded:")
            for doc in docs:
                print(doc.metadata)
            break  # Exit the loop once the valid URL is processed
        else:
            print("Invalid GitHub URL. Please try again.")
            github_url = input("Please enter the GitHub repository URL: ")

    print("Uploading to vector store...")

    # ====== Create vector store and upload data ======

    vector_store = DeepLakeVectorStore(
        #dataset_path=userdata.get('dataset_path'),
        dataset_path='hub://your_org/repository_db',
        overwrite=True,
        runtime={"tensor_db": True},
    )

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
    query_engine = index.as_query_engine()

    # Include a simple question to test.
    intro_question = "What is the repository about?"
    print(f"Test question: {intro_question}")
    print("=" * 50)
    answer = query_engine.query(intro_question)

    print(f"Answer: {textwrap.fill(str(answer), 100)} \n")
    while True:
        user_question = input("Please enter your question (or type 'exit' to quit): ")
        if user_question.lower() == "exit":
            print("Exiting, thanks for chatting!")
            break

        print(f"Your question: {user_question}")
        print("=" * 50)

        answer = query_engine.query(user_question)
        print(f"Answer: {textwrap.fill(str(answer), 100)} \n")


if __name__ == "__main__":
    main()

# Environment

In [ ]:
!pip list